<a href="https://colab.research.google.com/github/akumarpandey686/brain_tumor_classification/blob/master/Project_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [0]:
IMAGE_SIZE = [224, 224]

In [0]:
#Give dataset path
train_path = '/content/drive/My Drive/MTech/final_dataset/train'
test_path = '/content/drive/My Drive/MTech/final_dataset/test'

In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [9]:
# useful for getting number of classes
folders = glob('/content/drive/My Drive/MTech/final_dataset/train/*')
print(len(folders))

2


In [10]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [15]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/My Drive/MTech/final_dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 188 images belonging to 2 classes.


In [16]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/MTech/final_dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 75 images belonging to 2 classes.


In [17]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/10
5/5 [==============================] - 100s 20s/step - loss: 1200.1843 - accuracy: 0.5375 - val_loss: 1902.9099 - val_accuracy: 0.6007

Epoch 00001: val_loss improved from inf to 1902.90991, saving model to mymodel.h5
Epoch 2/10
5/5 [==============================] - 18s 4s/step - loss: 980.4932 - accuracy: 0.6776 - val_loss: 142.3097 - val_accuracy: 0.7554

Epoch 00002: val_loss improved from 1902.90991 to 142.30974, saving model to mymodel.h5
Epoch 3/10
5/5 [==============================] - 14s 3s/step - loss: 481.3989 - accuracy: 0.7437 - val_loss: 1915.4915 - val_accuracy: 0.5889

Epoch 00003: val_loss did not improve from 142.30974
Epoch 4/10
5/5 [==============================] - 14s 3s/step - loss: 576.6963 - accuracy: 0.7961 - val_loss: 454.4171 - val_accuracy: 0.7617

Epoch 00004: val_loss did not improve from 142.30974
Epoch 5/10
5/5 [==============================] - 14s 3s/step - loss: 432.2378 - accuracy: 0.7875 - val_loss: 1275.9288 - val_accuracy: 0.6974

Epo